In [ ]:
%pylab inline
from __future__ import print_function, division
from statsmodels.kernel_methods import kde, kernels, fast_linbin, kde_methods, bandwidths, bootstrap
from statsmodels.kernel_methods.kde_utils import Grid, GridInterpolator
from scipy import integrate, stats
from matplotlib.font_manager import FontProperties

## Definition of a 2D distribution

In [ ]:
N = 1e4
d1x = stats.norm(loc=1, scale=1)
d1y = stats.norm(loc=-2.5, scale=0.2)
d2x = stats.norm(loc=-2, scale=0.5)
d2y = stats.norm(loc=1, scale=1.1)
d1 = c_[d1x.rvs(N), d1y.rvs(N)]
d2 = c_[d2x.rvs(N), d2y.rvs(N)]
d = concatenate((d1, d2), axis=0)
k = kde.KDE(d, method=kde_methods.Cyclic)
print(d.min(axis=0), d.max(axis=0))
def real_pdf(xs, ys):
    return ((d1x.pdf(xs) * d1y.pdf(ys)) + (d2x.pdf(xs) * d2y.pdf(ys)))/2

### Finding the bounds for drawing

In [ ]:
xmin = floor(d[:,0].min())-1
xmax = ceil(d[:,0].max())+1
ymin = floor(d[:,1].min())-1
ymax = ceil(d[:,1].max())+1
print('Extent: x = {0:g} -- {1:g}, y = {2:g} -- {3:g}'.format(xmin, xmax, ymin, ymax))
est = k.fit()
print('bw = {0}, det_inv_bw = {1}'.format(est.bandwidth, est.det_inv_bandwidth))

### Estimation using diagonal Scott's rule of thumb

In [ ]:
%%time
GN = 128
grid = Grid.fromSparse(ogrid[xmin:xmax:GN*1j, ymin:ymax:GN*1j])
pdf = est(grid.linear())
pdf.shape = grid.shape

In [ ]:
%%time
print("For comparison, estimation using gaussian_kde:")
GN = 128
grid = Grid.fromSparse(ogrid[xmin:xmax:GN*1j, ymin:ymax:GN*1j])
est_st = stats.gaussian_kde(d.T)
pdf_st = est_st(grid.linear().T)

In [ ]:
mesh, pdf2 = est.grid([2**9, 2**10])
print(pdf2.shape, sum(pdf2)*prod(mesh.start_interval))
mesh[0].shape, mesh[1].shape

In [ ]:
%timeit est.grid(2**10)

In [ ]:
dx = grid.start_interval[0]
dy = grid.start_interval[1]
da = dx*dy
print('dx: {0:.4g}, dy: {1:.4g}, da: {2:.4g}'.format(dx, dy, da))
print('sum: {:.4g}'.format(sum(pdf)*da))

### The Real Distribution

In [ ]:
rxs, rys = mgrid[xmin:xmax:210j, ymin:ymax:201j]
f = figure()
f.set_size_inches(12, 8)
pcolormesh(rxs, rys, real_pdf(rxs, rys), cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()

### Drawing the result

In [ ]:
gr = grid.full()
f = figure()
f.set_size_inches(24, 8)
ax1 = f.add_subplot(1,2,1)
pcolormesh(gr[...,0], gr[...,1], pdf, cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Estimation of bi-modal distribution (direct)')
ax1 = f.add_subplot(1,2,2)
m2 = mesh.full('C')
pcolormesh(m2[0], m2[1], pdf2, cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Estimation of bi-modal distribution (FFT)')

**Note:** The red lines show the center of the reference system, the green lines the center of the two modes.
Compared to the real, expected, distribution, the result is not great! The peaks are too broad, and of similar width and height.

In [ ]:
m2g = Grid.fromFull(m2[:,::2**4,::2**5], 'C')
pdf3 = est.pdf(m2g.linear())
pdf3.shape = m2g.shape
np.max(abs(pdf2[::2**4,::2**5] - pdf3))

In [ ]:
pcolormesh(m2[0,::2**4,::2**5], m2[1,::2**4,::2**5], pdf2[::2**4,::2**5]-pdf3, cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Comparing FFT and Direct computation')
m2[0,::2**4,::2**5].shape

## Bootstrapping for Confidence Interval estimation

In [ ]:
bs_grid, bs_values = bootstrap.bootstrap_grid(est, 1000, (0.95, 0.99))
_, small_values = est.grid()

In [ ]:
bs_values.shape, small_values.shape

In [ ]:
gr = bs_grid.full()
f = figure()
f.set_size_inches(24, 8)
ax1 = f.add_subplot(1,3,2)
pcolormesh(gr[...,0], gr[...,1], small_values, cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Estimation of bi-modal distribution')
ax2 = f.add_subplot(1,3,1)
pcolormesh(gr[...,0], gr[...,1], bs_values[...,0,0], cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Low CI (95%)')
ax2 = f.add_subplot(1,3,3)
pcolormesh(gr[...,0], gr[...,1], bs_values[...,0,1], cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('High CI (95%)')


## Using cross-validation to estimate bandwidth

In [ ]:
k1 = k.copy()
bw = kde.bandwidths.crossvalidation(kde.bandwidths.CV_IMSE, grid_size=256, folding=10, use_grid=True)
k1.bandwidth = bw

In [ ]:
imse = kde.bandwidths.CV_IMSE(k1, grid_size=256, folding=10, use_grid=True)

In [ ]:
print('imse(0.02) :', imse([0.02, 0.02]))
print('imse(0.2) :', imse([0.2, 0.2]))
print('imse(0.4) :', imse([0.4, 0.4]))
print('imse(0.6) :', imse([0.6, 0.6]))

In [ ]:
imse.test_est.bandwidth = imse.LSO_est.bandwidth = 0.2
test_gr, test_p = imse.test_est.grid(256)
LSO_gr, LSO_p = imse.LSO_est.grid(256)
LSO_interp = GridInterpolator(LSO_gr, LSO_p)
vals = LSO_interp(est.exog)
val2 = imse.LSO_est(est.exog)
2*vals.sum()/1000, 2*val2.sum()/1000, test_gr.integrate(test_p**2)

In [ ]:
f=figure()
f.set_size_inches(16,12)
ax1 = f.add_subplot(2,2,1)
tripcolor(est.exog[:,0], est.exog[:,1], vals, shading='gouraud')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Grid estimate')
ax2 = f.add_subplot(2,2,2)
tripcolor(est.exog[:,0], est.exog[:,1], val2, shading='gouraud')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Direct Calculation')
ax3 = f.add_subplot(2,2,3)
tripcolor(est.exog[:,0], est.exog[:,1], vals-val2, shading='gouraud')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('difference')
ax4 = f.add_subplot(2,2,4)
LSO_grd = LSO_gr.full()
pcolormesh(LSO_grd[...,0], LSO_grd[...,1], LSO_p, cmap=cm.jet, shading='gouraud')
axis('equal')
xlim(xmin, xmax)
ylim(ymin, ymax)
colorbar()
title('Grid values')
f.suptitle('Evaluation of the Grid approximation', fontsize=rcParams['axes.titlesize'], fontproperties=FontProperties(weight='bold'))

In [ ]:
%%time
est1 = k1.fit()

In [ ]:
print('est: ', est.bandwidth)
print('est1: ', est1.bandwidth)
optim = est1.bandwidth

In [ ]:
xxs, pdf3 = est1.grid(512)
xxs = Grid(xxs).full('C')

In [ ]:
f = figure()
f.set_size_inches(24, 8)
ax1 = f.add_subplot(1,2,1)
pcolormesh(xxs[0], xxs[1], pdf3, cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xxs[0].min(), xxs[0].max())
ylim(xxs[1].min(), xxs[1].max())
title('Estimated distribution')
colorbar()
ax2 = f.add_subplot(1,2,2)
pcolormesh(xxs[0], xxs[1], real_pdf(xxs[0], xxs[1]), cmap=cm.jet, shading='gouraud')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
xlim(xxs[0].min(), xxs[0].max())
ylim(xxs[1].min(), xxs[1].max())
title('Real distribution')
colorbar()

**Notes:** The estimation is much better: the peak's relative heights are correct and so are their widths!

And a few values for the IMSE:

In [ ]:
imse = bandwidths.ContinuousIMSE(k, grid_size = 256, folding=10, use_grid=True)
print('Optimum: ', imse(est1.bandwidth))
print('Double Scott: ', imse(2*est.bandwidth))
print('Scott: ', imse(est.bandwidth))
print('Half Scott: ', imse(est.bandwidth/2))
print('0.2, 0.1: ', imse([0.2, 0.1]))
print('0.02, 0.01: ', imse([0.02, 0.01]))

In [ ]:
print('folding\n-------')
imse = bandwidths.ContinuousIMSE(k, grid_size = 512, folding=10, use_grid = True)
%time imse(est1.bandwidth)
print('\nLOO\n---')
imse = bandwidths.ContinuousIMSE(k, grid_size = 512)
%time imse(est1.bandwidth)
print('\nLOO Sampling\n------------')
imse = bandwidths.ContinuousIMSE(k, grid_size = 512, sampling=1000)
%time imse(est1.bandwidth)
None # Suppress output

### To check : the 2D histogram

In [ ]:
mesh, H = _fast_linbin.fast_linbin_nd(d, [[xmin, xmax], [ymin, ymax]], M=128)
H /= sum(H)*prod(mesh.start_volume)
mesh

In [ ]:
f = figure()
f.set_size_inches(8,7)
xxh = mesh.full('C')
pcolormesh(xxh[0], xxh[1], H, cmap=cm.jet, shading='flat')
axhline(0, color='red', linewidth=2, linestyle='--')
axvline(0, color='red', linewidth=2, linestyle='--')
axhline(-2.5, color='green', linewidth=2, linestyle=':')
axvline(1, color='green', linewidth=2, linestyle=':')
axhline(1, color='green', linewidth=2, linestyle=':')
axvline(-2, color='green', linewidth=2, linestyle=':')
axis('equal')
colorbar()
xlim(xmin, xmax)
ylim(ymin, ymax)

## Cyclic distribution

In [ ]:
N=1e4
dcx = stats.norm(loc=2, scale=.5)
dcy = stats.norm(loc=3, scale=.3)
rx = mod(dcx.rvs(N), 2)
ry = mod(dcy.rvs(N), 3)
d = c_[rx, ry]
bounds = array([[0,2], [0,3]])
d.shape

In [ ]:
mesh, H = _fast_linbin.fast_linbin_nd(d, bounds, M=128)
print(mesh.start_volume)
H /= sum(H)*mesh.start_volume

In [ ]:
print(bounds.flatten())
f = figure()
f.set_size_inches(8,7)
xxh = mesh.full('C')
imshow(H, extent=bounds.flatten(), cmap=cm.jet)
axis('equal')
colorbar()
xlim(*bounds[0])
ylim(*bounds[1])

In [ ]:
%%time
k = kde.KDE(d, lower=bounds[:,0], upper=bounds[:,1], method=kde_methods.Cyclic())
k.bandwidth = bandwidths.lsq_crossvalidation(imse_args=dict(folding=10, use_grid=True, grid_size=512))
est = k.fit()

In [ ]:
mesh, pdf = est.grid(N=256)
print(mesh, ' -- bw = ', est.bandwidth)
print("sum: ", mesh.integrate(pdf))

In [ ]:
print(bounds.flatten())
f = figure()
f.set_size_inches(8,7)
imshow(pdf, extent=mesh.bounds.flatten(), cmap=cm.jet)
axis('equal')
colorbar()
xlim(*bounds[0])
ylim(*bounds[1])